In [1]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/spaceship-titanic/sample_submission.csv
/kaggle/input/spaceship-titanic/train.csv
/kaggle/input/spaceship-titanic/test.csv


In [2]:
datapath = '/kaggle/input/spaceship-titanic/train.csv'
datap = '/kaggle/input/spaceship-titanic/test.csv'
df = pd.read_csv(datapath)
df

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [3]:
df.isnull().sum()

PassengerId       0
HomePlanet      201
CryoSleep       217
Cabin           199
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Name            200
Transported       0
dtype: int64

In [4]:
for column in df.select_dtypes(include='object').columns:
    df[column].fillna(df[column].mode()[0], inplace=True)

In [5]:
for column in df.select_dtypes(include='number').columns:
    df[column].fillna(df[column].mean(), inplace=True)

In [6]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
ohe_result = ohe.fit_transform(df[['HomePlanet']]).toarray()
ohe_df = pd.DataFrame(ohe_result, columns=ohe.get_feature_names_out(['HomePlanet']))

df_encoded = pd.concat([df, ohe_df], axis=1)

ohe_result2 = ohe.fit_transform(df[['Destination']]).toarray()
ohe_df2 = pd.DataFrame(ohe_result2, columns=ohe.get_feature_names_out(['Destination']))
df_encoded = pd.concat([df_encoded, ohe_df2], axis=1)

In [7]:
df_encoded['CryoSleepE'] = df_encoded['CryoSleep'].astype(int)
df_encoded['VIPE'] = df_encoded['VIP'].astype(int)
df_encoded['TransportedE'] = df_encoded['Transported'].astype(int)

In [8]:
df_encoded[['CabinDeck', 'CabinNum', 'CabinSide']] = df_encoded['Cabin'].str.split('/', expand=True)

In [9]:
label_encoder = LabelEncoder()
df_encoded['CabinSide'] = label_encoder.fit_transform(df_encoded['CabinSide'])
df_encoded['CabinDeck'] = label_encoder.fit_transform(df_encoded['CabinDeck'])
df_encoded['Name'] = label_encoder.fit_transform(df_encoded['Name'])
df_encoded

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,...,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,CryoSleepE,VIPE,TransportedE,CabinDeck,CabinNum,CabinSide
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0,0,0,1,0,0
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,...,0.0,0.0,0.0,1.0,0,0,1,5,0,1
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,...,0.0,0.0,0.0,1.0,0,1,0,0,0,1
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,...,0.0,0.0,0.0,1.0,0,0,0,0,0,1
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,...,0.0,0.0,0.0,1.0,0,0,1,5,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,...,0.0,1.0,0.0,0.0,0,1,0,0,98,0
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1,0,0,6,1499,1
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,...,0.0,0.0,0.0,1.0,0,0,1,6,1500,1
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,...,0.0,1.0,0.0,0.0,0,0,0,4,608,1


In [10]:
columns_to_remove = ['HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'VIP', 'Transported']
df1 = df_encoded.drop(columns=columns_to_remove)


In [11]:
df1

,PassengerId,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,CryoSleepE,VIPE,TransportedE,CabinDeck,CabinNum,CabinSide
0,0001_01,39.0,0.0,0.0,0.0,0.0,0.0,5252,0.0,1.0,0.0,0.0,0.0,1.0,0,0,0,1,0,0
1,0002_01,24.0,109.0,9.0,25.0,549.0,44.0,4502,1.0,0.0,0.0,0.0,0.0,1.0,0,0,1,5,0,1
2,0003_01,58.0,43.0,3576.0,0.0,6715.0,49.0,457,0.0,1.0,0.0,0.0,0.0,1.0,0,1,0,0,0,1
3,0003_02,33.0,0.0,1283.0,371.0,3329.0,193.0,7149,0.0,1.0,0.0,0.0,0.0,1.0,0,0,0,0,0,1
4,0004_01,16.0,303.0,70.0,151.0,565.0,2.0,8319,1.0,0.0,0.0,0.0,0.0,1.0,0,0,1,5,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,41.0,0.0,6819.0,0.0,1643.0,74.0,3524,0.0,1.0,0.0,1.0,0.0,0.0,0,1,0,0,98,0
8689,9278_01,18.0,0.0,0.0,0.0,0.0,0.0,4780,1.0,0.0,0.0,0.0,1.0,0.0,1,0,0,6,1499,1
8690,9279_01,26.0,0.0,0.0,1872.0,1.0,0.0,3002,1.0,0.0,0.0,0.0,0.0,1.0,0,0,1,6,1500,1
8691,9280_01,32.0,0.0,1049.0,0.0,353.0,3235.0,1596,0.0,1.0,0.0,1.0,0.0,0.0,0,0,0,4,608,1


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier


X = df1.drop('TransportedE', axis=1)
y = df1['TransportedE']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [14]:
y_pred = rf_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
accuracy

0.7883841288096607